<a href="https://colab.research.google.com/github/EdoLeta/Amharic-Translation/blob/main/Edosa%20Leta_English_to_Amharic_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# TODO: Set your source and target languages. Keep in mind, these traditionally use language codes as found here:
# These will also become the suffix's of all vocab and corpus files used throughout
import os
source_language = "en"
target_language = "am"
tag = "baseline" # Give a unique name to your folder - this is to ensure you don't rewrite any models you've already submitted

os.environ["src"] = source_language # Sets them in bash as well, since we often use bash scripts
os.environ["tgt"] = target_language
os.environ["tag"] = tag

# This will save it to a folder in our gdrive instead!
!mkdir -p "/content/drive/My Drive/masakhane/$src-$tgt"
os.environ["gdrive_path"] = "/content/drive/My Drive/masakhane/%s-%s" % (source_language, target_language)

In [ ]:
# I am training the data with another dataset that I created have created for this specific domain
from google.colab import files
uploaded = files.upload()

Saving en_am_data.csv to en_am_data.csv


In [ ]:
import pandas as pd
import io
data = pd.read_csv(io.BytesIO(uploaded['en_am_data.csv']))
# Dataset is now stored in a Pandas Dataframe

In [ ]:
data.head()

,English,Amharic
0,"17 When Aʹbram was 99 years old, Jehovah appe...",17 አብራም ዕድሜው 99 ዓመት ሲሆን ይሖዋ ተገለጠለትና እንዲህ አለው፦...
1,2 I will establish my covenant between me and...,2 ከአንተ ጋር ቃል ኪዳኔን እመሠርታለሁ፤+ አንተንም እጅግ አበዛሃለሁ።”+
2,"3 At this Aʹbram fell facedown, and God conti...",3 በዚህ ጊዜ አብራም በግንባሩ ተደፋ፤ አምላክም እንዲህ በማለት እሱን ...
3,"4 “As for me, look! my covenant is with you,+...",4 “በእኔ በኩል ከአንተ ጋር የገባሁት ቃል ኪዳኔ እንደጸና ነው፤+ አን...
4,5 Your name will no longer be Aʹbram;* your n...,5 ከእንግዲህ ስምህ አብራም* አይባልም፤ የብዙ ብሔር አባት ስለማደርግህ...


In [ ]:
data[data.duplicated()]

,English,Amharic
498,"17 When Aʹbram was 99 years old, Jehovah appe...",17 አብራም ዕድሜው 99 ዓመት ሲሆን ይሖዋ ተገለጠለትና እንዲህ አለው፦...
500,"3 At this Aʹbram fell facedown, and God conti...",3 በዚህ ጊዜ አብራም በግንባሩ ተደፋ፤ አምላክም እንዲህ በማለት እሱን ...
501,"4 “As for me, look! my covenant is with you,+...",4 “በእኔ በኩል ከአንተ ጋር የገባሁት ቃል ኪዳኔ እንደጸና ነው፤+ አን...
502,5 Your name will no longer be Aʹbram;* your n...,5 ከእንግዲህ ስምህ አብራም* አይባልም፤ የብዙ ብሔር አባት ስለማደርግህ...
504,7 “And I will keep my covenant between me and...,7 “ለአንተም ሆነ ከአንተ በኋላ ለሚመጡት ዘሮችህ አምላክ እሆን ዘንድ ...
...,...,...
891,9 After that Jacob said: “O God of my father ...,9 ከዚህ በኋላ ያዕቆብ እንዲህ አለ፦ “‘ወደ አገርህና ወደ ዘመዶችህ ተ...
892,10 I am unworthy of all the loyal love and of...,10 እኔ ባሪያህ ይህን ያህል ታማኝ ፍቅርና ታማኝነት ልታሳየኝ+ የሚገባ...
893,"11 Save me, I pray you,+ from the hand of my ...",11 ከወንድሜ ከኤሳው እጅ እንድታድነኝ እለምንሃለሁ፤+ ምክንያቱም መጥቶ...
895,13 And he spent the night there. Then he took...,13 እሱም በዚያ ሌሊት እዚያው አደረ። ከንብረቱም መካከል የተወሰነውን ...


In [ ]:
data = data.rename(columns={"English":"source_sentence", "Amharic":"target_sentence"})

In [ ]:
print("Length of Data before Removing duplicate: ",len(data))
data = data.drop_duplicates()
print("Length of Data after Removing duplicate: ",len(data))

Length of Data before Removing duplicate:  998
Length of Data after Removing duplicate:  679


In [ ]:
# Do the split between dev/test/train and create parallel corpora
num_dev_patterns = 1000
num_test_patterns = 1000
df = data
# Lower case the corpora
df["source_sentence"] = df["source_sentence"].str.lower()
df["target_sentence"] = df["target_sentence"].str.lower()


devtest = df.tail(num_dev_patterns + num_test_patterns)
test = devtest.tail(num_test_patterns)
dev = devtest.head(num_dev_patterns)
# stripped = df.drop(df.tail(num_dev_patterns + num_test_patterns).index)

stripped = df.head(4096)

stripped[["source_sentence"]].to_csv("train.en", index=False)
stripped[["target_sentence"]].to_csv("train.am", index=False)

dev[["source_sentence"]].to_csv("dev.en", index=False)
dev[["target_sentence"]].to_csv("dev.am", index=False)

test[["source_sentence"]].to_csv("test.en", index=False)
test[["target_sentence"]].to_csv("test.am", index=False)


In [ ]:
# train_new = df.head(4096)

In [ ]:
# Install JoeyNMT
! git clone https://github.com/joeynmt/joeynmt.git
! cd joeynmt; pip3 install .

Cloning into 'joeynmt'...
remote: Enumerating objects: 78, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (61/61), done.
remote: Total 3021 (delta 36), reused 42 (delta 17), pack-reused 2943
Receiving objects: 100% (3021/3021), 2.94 MiB | 30.12 MiB/s, done.
Resolving deltas: 100% (2057/2057), done.
Processing /content/joeynmt
     |████████████████████████████████| 15.3MB 220kB/s 
     |████████████████████████████████| 61kB 9.4MB/s 
     |████████████████████████████████| 645kB 55.3MB/s 
     |████████████████████████████████| 348kB 53.2MB/s 
     |████████████████████████████████| 112kB 57.8MB/s 
     |████████████████████████████████| 225kB 57.7MB/s 
     |████████████████████████████████| 61kB 5.5MB/s 
     |████████████████████████████████| 747kB 55.2MB/s 
  Created wheel for joeynmt: filename=joeynmt-1.3-cp37-none-any.whl size=83160 sha256=a5d66fbf765d7cf492be778860d889f8d292a6fdeb23539fde355290f184c9e0
  Stored in directory: /tmp/pip-ephem-

In [ ]:
# One of the huge boosts in NMT performance was to use a different method of tokenizing. 
# Usually, NMT would tokenize by words. However, using a method called BPE gave amazing boosts to performance

# Do subword NMT
! mkdir joeynmt/data/
! mkdir joeynmt/data/enam/
! export data_path=joeynmt/data/$src$tgt/
! subword-nmt learn-joint-bpe-and-vocab --input train.$src train.$tgt -s 4000 -o bpe.codes.4000 --write-vocabulary vocab.$src vocab.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < train.$src > train.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < train.$tgt > train.bpe.$tgt

! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < dev.$src > dev.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < dev.$tgt > dev.bpe.$tgt
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$src < test.$src > test.bpe.$src
! subword-nmt apply-bpe -c bpe.codes.4000 --vocabulary vocab.$tgt < test.$tgt > test.bpe.$tgt

# Create directory, move everyone we care about to the correct location
#! mkdir -p $data_path
! cp train.* joeynmt/data/enam/
! cp test.* joeynmt/data/enam/
! cp dev.* joeynmt/data/enam/
! cp bpe.codes.4000 $data_path
! ls $data_path

# Create that vocab using build_vocab
! sudo chmod 777 joeynmt/scripts/build_vocab.py
! joeynmt/scripts/build_vocab.py joeynmt/data/$src$tgt/train.bpe.$src joeynmt/data/$src$tgt/train.bpe.$tgt --output_path joeynmt/data/$src$tgt/vocab.txt

# Some output
! echo "BPE amharic Sentences"
! tail -n 5 test.bpe.$tgt
! echo "Combined BPE Vocab"
! tail -n 10 joeynmt/data/enam/vocab.txt


cp: missing destination file operand after 'bpe.codes.4000'
Try 'cp --help' for more information.
bpe.codes.4000	dev.en		sample_data  test.en	   train.en
dev.am		drive		test.am      train.am	   vocab.am
dev.bpe.am	en_am_data.csv	test.bpe.am  train.bpe.am  vocab.en
dev.bpe.en	joeynmt		test.bpe.en  train.bpe.en
BPE amharic Sentences
5 አብራም ሚስቱን ሦ@@ ራ@@ ን@@ ፣+ የ@@ ወንድ@@ ሙ@@ ን ልጅ ሎጥ@@ ን+ እንዲሁም ያ@@ ፈ@@ ሩትን ንብረት ሁሉ@@ ና+ በ@@ ካራ@@ ን አብረ@@ ዋ@@ ቸው የነበሩትን ሰዎ@@ ች@@ * በሙሉ ይዞ ተነሳ፤ እነሱም ወደ ከነአን ምድር አቀ@@ ኑ።+ ከነአን ምድር በ@@ ደረ@@ ሱም ጊዜ
6 አብራም በ@@ ሞ@@ ሬ ትላልቅ ዛፎ@@ ች+ አቅራቢያ እስከሚ@@ ገኘው ሴኬ@@ ም+ እስከሚ@@ ባለው አካባቢ ድረስ ወደ ምድሪቱ ዘ@@ ል@@ ቆ ገባ። በዚ@@ ያን ጊዜ ከነአናውያን በ@@ ምድሪቱ ይኖ@@ ሩ ነበር።
7 ይሖዋም ለ@@ አብራም ተገለጠ@@ ለት@@ ና “@@ ይህችን ምድር ለ@@ ዘር@@ ህ+ እሰጣ@@ ለሁ@@ ”+ አለው። ስለዚህ አብራም ተገልጦ@@ ለት ለ@@ ነበረው ለ@@ ይሖዋ በዚያ መሠዊያ ሠራ@@ ።
8 በኋላም ከዚያ ተነስቶ ከ@@ ቤቴል+ በስተ ምሥ@@ ራቅ ወደሚገኘው ተራራማ አካባቢ ተ@@ ጓዘ@@ ፤ እሱም ቤቴ@@ ልን በስተ ም@@ ዕ@@ ራ@@ ብ@@ ፣ ጋ@@ ይ@@ ን+ ደግሞ በስተ ምሥ@@ ራቅ አድርጎ ድንኳኑን ተከ@@ ለ@@ ። በዚያም ለ@@ ይሖዋ መሠዊያ ሠራ@@ ፤+ የ@@ ይሖዋ@@ ንም ስም ጠራ@@ ።+
9 ከዚያ በኋላ አብራም ድ

In [ ]:
# Also move everything we care about to a mounted location in google drive (relevant if running in colab) at gdrive_path
! cp train.* "$gdrive_path"
! cp test.* "$gdrive_path"
! cp dev.* "$gdrive_path"
! cp bpe.codes.4000 "$gdrive_path"
! ls "$gdrive_path"

bpe.codes.4000	dev.bpe.en  test.am	 test.en       train.bpe.en
dev.am		dev.en	    test.bpe.am  train.am      train.en
dev.bpe.am	models	    test.bpe.en  train.bpe.am


In [ ]:
def get_last_checkpoint(directory):
  last_checkpoint = ''
  for filename in os.listdir(directory):
    if not 'best' in filename and filename.endswith(".ckpt"):
        if not last_checkpoint or int(filename.split('.')[0]) > int(last_checkpoint.split('.')[0]):
          last_checkpoint = filename
  return last_checkpoint

In [ ]:
g_drive_path = "/content/drive/MyDrive/masakhane/en-am"
# g_drive_path = "/content/drive/My Drive/masakhane/%s-%s-%s" % (source_language, target_language, tag)
print(g_drive_path)

/content/drive/MyDrive/masakhane/en-am


In [ ]:
models_path = "/content/drive/MyDrive/masakhane/en-am/models"
print(models_path)

/content/drive/MyDrive/masakhane/en-am/models


In [ ]:
# Copy the created models from the temporary storage to main storage on google drive for persistant storage 
# the content of te folder will be overwrite when you start trainin
# !cp -r "/content/drive/MyDrive/masakhane/en-am/models"* "$gdrive_path/models/${src}${tgt}_transformer/"
last_checkpoint = get_last_checkpoint(models_path)
print('Last checkpoint :',last_checkpoint)

Last checkpoint : 5200.ckpt


In [ ]:
!mkdir -p /content/joeynmt/configs/models

In [ ]:
# This creates the config file for our JoeyNMT system

name = '%s%s' % (source_language, target_language)

config = """
name: "{name}_transformer"

data:
    src: "{source_language}"
    trg: "{target_language}"
    train: "data/{name}/train.bpe"
    dev:   "data/{name}/dev.bpe"
    test:  "data/{name}/test.bpe"
    level: "bpe"
    lowercase: False
    max_sent_length: 100
    src_vocab: "data/{name}/vocab.txt"
    trg_vocab: "data/{name}/vocab.txt"

testing:
    beam_size: 5
    alpha: 1.0

training:
    load_model: "/content/drive/MyDrive/masakhane/en-am/models/best.ckpt" # if given, load a pre-trained model from this checkpoint
    reset_optimizer: True
    reset_best_ckpt: True
    reset_scheduler: True
    random_seed: 42
    optimizer: "adam"
    normalization: "tokens"
    adam_betas: [0.9, 0.999] 
    scheduling: "noam"            # Try switching from plateau to Noam scheduling
    learning_rate_factor: 0.5       # factor for Noam scheduler (used with Transformer)
    learning_rate_warmup: 1000      # warmup steps for Noam scheduler (used with Transformer)
    patience: 8
    decrease_factor: 0.7
    loss: "crossentropy"
    learning_rate: 0.0002
    learning_rate_min: 0.00000001
    weight_decay: 0.0
    label_smoothing: 0.1
    batch_size: 4560
    batch_type: "token"
    eval_batch_size: 3600
    eval_batch_type: "token"
    batch_multiplier: 1
    early_stopping_metric: "ppl"
    epochs: 5 # TODO: Decrease for when playing around and checking of working. Around 30 is sufficient to check if its working at all
    validation_freq: 50 # Decrease this for testing(it was 400 at first)
    logging_freq: 100
    eval_metric: "bleu"
    model_dir: "models/{name}_transformer"
    overwrite: True
    shuffle: True
    use_cuda: True
    max_output_length: 100
    print_valid_sents: [0, 1, 2, 3]
    keep_last_ckpts: 3

model:
    initializer: "xavier"
    bias_initializer: "zeros"
    init_gain: 1.0
    embed_initializer: "xavier"
    embed_init_gain: 1.0
    tied_embeddings: True
    tied_softmax: True
    encoder:
        type: "transformer"
        num_layers: 6
        num_heads: 8
        embeddings:
            embedding_dim: 512
            scale: True
            dropout: 0.
        # typically ff_size = 4 x hidden_size
        hidden_size: 512
        ff_size: 2048
        dropout: 0.3
    decoder:
        type: "transformer"
        num_layers: 6
        num_heads: 8
        embeddings:
            embedding_dim: 512
            scale: True
            dropout: 0.
        # typically ff_size = 4 x hidden_size
        hidden_size: 512
        ff_size: 2048
        dropout: 0.3
""".format(name=name, source_language=source_language, target_language=target_language)
with open("/content/joeynmt/configs/models/transformer_{name}.yaml".format(name=name),'w') as f:
    f.write(config)

In [ ]:
!cp "/content/drive/MyDrive/masakhane/en-am/models/enam_transformer/best.ckpt" "/content/joeynmt/configs/models"

In [ ]:
!cd joeynmt; python3 -m joeynmt train configs/models/transformer_$src$tgt.yaml

2021-03-17 23:56:08,664 - INFO - root - Hello! This is Joey-NMT (version 1.3).
2021-03-17 23:56:08,727 - INFO - joeynmt.data - Loading training data...
2021-03-17 23:56:08,737 - INFO - joeynmt.data - Building vocabulary...
2021-03-17 23:56:09,008 - INFO - joeynmt.data - Loading dev data...
2021-03-17 23:56:09,017 - INFO - joeynmt.data - Loading test data...
2021-03-17 23:56:09,026 - INFO - joeynmt.data - Data loaded.
2021-03-17 23:56:09,026 - INFO - joeynmt.model - Building an encoder-decoder model...
2021-03-17 23:56:09,879 - INFO - joeynmt.model - Enc-dec model built.
2021-03-17 23:56:09.997846: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0
2021-03-17 23:56:11,252 - INFO - joeynmt.training - Total params: 46265344
2021-03-17 23:56:21,764 - INFO - joeynmt.training - Loading model from /content/drive/MyDrive/masakhane/en-am/models/best.ckpt
Traceback (most recent call last):
  File "/usr/lib/python3.7/runpy.py", li

In [ ]:
! cat joeynmt/models/enam_transformer/validations.txt

cat: joeynmt/models/enam_transformer/validations.txt: No such file or directory


In [ ]:
# Copy the created models from the notebook storage to google drive for persistant storage 
!mkdir "$gdrive_path/models/"
!cp -r joeynmt/models/* "$gdrive_path/models/${src}${tgt}_transformer/"

mkdir: cannot create directory ‘/content/drive/My Drive/masakhane/en-am/models/’: File exists


In [ ]:
! cat "$gdrive_path/models/${src}${tgt}_transformer/validations.txt"

cat: '/content/drive/My Drive/masakhane/en-am/models/enam_transformer/validations.txt': No such file or directory


In [ ]:
!cp "/content/drive/MyDrive/masakhane/en-am/models/enam_transformer/5200.ckpt" "/content/joeynmt/models/enam_transformer/"


In [ ]:
! cd joeynmt; python3 -m joeynmt test models/enam_transformer/config.yaml

Traceback (most recent call last):
  File "/usr/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/content/joeynmt/joeynmt/__main__.py", line 41, in <module>
    main()
  File "/content/joeynmt/joeynmt/__main__.py", line 32, in main
    output_path=args.output_path, save_attention=args.save_attention)
  File "/content/joeynmt/joeynmt/prediction.py", line 267, in test
    cfg = load_config(cfg_file)
  File "/content/joeynmt/joeynmt/helpers.py", line 176, in load_config
    with open(path, 'r') as ymlfile:
FileNotFoundError: [Errno 2] No such file or directory: 'models/enam_transformer/config.yaml'


In [ ]:
cd joeynmt; python3 -m joeynmt translate models/enam_transformer/config.yaml < en.txt > enam_translate_output

[Errno 2] No such file or directory: 'joeynmt; python3 -m joeynmt translate models/enam_transformer/config.yaml < en.txt > enam_translate_output'
/content


In [ ]:
! cd joeynmt; python3 -m joeynmt translate models/enam_transformer/config.yaml < en.txt > enam_translate_output

/bin/bash: en.txt: No such file or directory


In [ ]:
! git clone https://github.com/joeynmt/joeynmt.git